# 2021 CCF BDCI基于飞桨实现花样滑冰选手骨骼点动作识别-第3名方案

## 赛题分析与理解

人体骨骼关键点近年来作为一种新颖的数据模态，被广泛的应用在各类动作识别任务中。由于骨骼关键点数据更加关注于动作本身、不包含背景信息的特点，该数据的引入提升了动作识别任务的准确率与鲁棒性。本赛题要求我们使用花样滑冰选手的骨骼关键点数据进行花样滑冰的动作识别，属于分类任务的范畴。不同于典型的动作识别任务，花样滑冰的动作粒度较细，同一大类下不同小类的动作之间区别较小，难以区分。同时，本赛题提供的数据集规模相对较小，存在类别不均衡、序列长度差异大的现象，如何处理这些问题是本赛题的关键。

## 模型构建思路与调优过程

由于骨骼关键点的序列数据天然具有时空图的结构，适用于作为图神经网络的输入，目前主流的基于骨骼关键点的动作识别算法大多基于图神经网络。相较于基于CNN与RNN的模型，图神经网络能够在较少的参数量下取得较高的识别精度。本次比赛中，我们的模型使用目前最优的骨骼关键点识别模型CTR-GCN[1]作为backbone，并使用带Label Smoothing的多分类Focal loss[2]作为损失函数，提升模型对困难样本的区分能力。在数据处理阶段，我们使用上采样策略缓解类别不均衡问题，利用分段随机采样的方法处理序列长度不均衡的问题，并引入了骨骼向量、骨骼角度、以及运动速度等多种特征特征训练模型，提升模型的准确率与鲁棒性。模型的框图如图1所示。

<div align="center">
  <img src='./work/doc/pipeline.png'>
  <br>图1 算法流程</br>
</div>


下面我们将对模型进行详细的介绍。

### Backbone模型

[CTR-GCN](https://arxiv.org/abs/2107.12213)是中科院自动化所发表在ICCV2021会议上的一个基于骨骼关键点的动作识别模型，其在学界的常用的几个基准数据集上均取得了目前最优的结果。我们使用该模型作为我们的backbone进行特征抽取，我们参考了论文作者提供的PyTorch版本代码[8]，并使用PaddlePaddle框架对其进行了复现，模型的超参数与原始代码一致。通过与基准模型[ST-GCN](https://arxiv.org/abs/1801.07455)[3]与官方提供的[AGCN](https://github.com/PaddlePaddle/PaddleVideo/blob/develop/docs/zh-CN/model_zoo/recognition/agcn.md)模型[4]对比，我们实现的CTR-GCN在仅使用骨骼关键点的2D坐标作为特征输入时的性能显著优于两个Baseline模型，实验结果见表1。

<div align="center">
  	<br>表1 不同Backbone模型的测试集准确率对比</br>
    <table>
        <tr>
            <th>Model</th>
            <th>ST-GCN</th>
            <th>AGCN</th>
            <th>CTR-GCN</th>
        </tr>
        <tr>
            <td>Test Acc</td>
            <td>60.83%</td>
            <td>62.74%</td>
            <td>65.76%</td>
        </tr>
    </table>
</div>

### 损失函数与训练策略

#### 1. 损失函数

通过对训练数据的分析，我们发现比赛中提供的训练集存在明显的类别不均衡的现象（见图2），同时由于花滑动作的小类之间难以区分，挖掘训练集中存在的困难样本并使得模型在这些样本上得到充分的训练有助于提升模型的分类准确率。出于以上考虑，我们使用的损失函数为带标签平滑的多分类Focal Loss，其形式如下
$$
L=-(1-p_t)^\gamma \sum_{c=1}^{K}y'_clog(p_c)$$
$$y'_c = \begin{cases}  \epsilon / K & c \neq t \\ 1 - \epsilon + \epsilon / K & c=t \end{cases}$$

其中$p_c$为模型预测的样本属于类别c的概率，$t$为样本的真实类别，$K$为类别数目，$y'_c$为经过label smoothing后第c类的类别标签，$\gamma$与$\epsilon \in [0, 1)$分别为控制Focal loss与Label Smoothing的超参数。由于Focal loss使用$(1-p_t)^\gamma$作为样本在loss中的权重，使得其能在训练过程中自动挖掘分类结果不佳的样本并提升其在loss中的占比，能够很好的缓解样本不均衡与困难样本挖掘这两个问题，提升模型的性能，同时，标签平滑的引入能够有效缓解过拟合现象。实验中我们取$\gamma=2.0$, $\epsilon=0.1$.

<div align="center">
    <img src='./work/doc/class_imbalance.png'>
  <br>图2 数据集的类别分布</br>
</div>

#### 2. 优化器选取

在训练策略的选择上，我们使用带Momentum的SGD算法作为模型的优化器，并使用带Warm-Up的余弦退火（Consine Annealing)[6]策略作为学习率衰减的方法。实验中我们发现逐epoch的学习率衰减性能优于逐iteration的学习率衰减策略，因此最终我们选用了逐epoch的余弦退火作为我们的学习率衰减策略，学习率随epoch的变化见图3。所有的实验中，我们取$epoch=90, max\_lr=0.1, start\_lr=0.01, momentum=0.9$。同时，为了进一步缓解模型的过拟合，我们对网络参数加上了幅度为4e-4的L2 weight decay项。

<div align="center">
    <img src='./work/doc/lr.png'>
  <br>图3 学习率衰减策略</br>
</div>

#### 3. 五折交叉验证与模型集成

由于官方数据不带有验证集，我们需要将数据集划分为训练集与验证集，以使用验证集进行模型评估。在模型的训练过程中，我们按照类别的分布将训练集等分为5个不相交的部分，每个部分的类别分布均与原始数据分布保持一致，进行5折交叉验证，产生5个效果最优的模型。后面我们将会介绍，每次训练时我们将在7组不同的特征组合上训练得到7个不同的模型，结合5折交叉验证，最终我们将得到35个在各自的验证集上表现最优的模型，这35个模型的集成构成了为我们最终的模型，多模型的集成显著提升了模型的准确率与鲁棒性。

### 数据处理

#### 1.上采样

如前所述，训练集中存在明显的类别不均衡的现象，可能导致模型在少样本的类别上训练不充分，降低模型在少样本类别上的准确率。针对这一问题，我们使用了上采样的策略。具体而言，我们对少样本的类别进行了随机重复，使得最终生成的训练集中各个类别的占比一致。结果显示，使用上采样后提升了模型的泛化能力，提升了模型在测试集上的准确率。我们在仅使用原始骨骼点坐标的集成模型上测试了使用Upsampling的效果，结果如表2所示。

<div align="center">
  <br>表2 是否使用Upsampling的测试集准确率对比</br>
    <table>
        <tr>
            <th> Setting (CTR-GCN-J)</th>
            <th> Test Acc</th>
        </tr>
        <tr>
            <td> w/o upsampling</td>
            <td> 67.35%</td>
        </tr>
        <tr>
            <td> w upsampling</td>
            <td> 68.63% </td>
        </tr>
    </table>
</div>

#### 2. 分段随机采样

对于序列识别任务，通常需要选取一个合适的序列长度作为模型的输入，并将所有输入样本的长度对齐至该选定的序列长度，所选择的序列长度以及样本长度的对齐方法往往能够显著影响模型性能。通过对训练集的观察我们发现，训练集中的关键点序列存在长度差异较大的情况，序列长度最大可达2037帧而最小仅有42帧，500帧以上的样本仅占27%，数据分布见图4。经过试验我们发现序列长度选取为350时效果较好,在验证集上不同帧的对比结果（模型集成前）见表3。对于长度小于350的样本，我们将其添加全0帧补全。对与长度大于350帧的样本，我们参考了论文[5]中提出的分段均匀采样方法，将其长度压缩至350帧。具体的做法为：将样本按照有效帧的长度划分为350个区间，在训练过程中，每次分别从每一段中随机采样一帧组成一个长度为350帧的样本作为模型的输入。

<div align="center">
    <img src='./work/doc/length_distribution.png'>
  <br>图4 样本的长度分布</br>
</div>

<div align="center">
  <br>表3 不同帧数的模型在验证集上的准确率对比</br>
    <table>
        <tr>
            <th> Frames </th>
            <th> 350 </th>
            <th> 400 </th>
            <th> 800 </th>
        </tr>
        <tr>
            <td> Val Acc </td>
            <td> 64.78% </td>
            <td> 63.41% </td>
            <td> 63.76% </td>
        </tr>
    </table>
</div>

#### 3. 高阶特征抽取

原始训练数据仅包含关键点的坐标与置信度，这属于一阶信息。尽管图神经网络具有抽取节点间高阶特征的能力，但是人为的引入与动作识别有关的高阶特征依然能够为网络提供更加丰富的信息，提升网络的性能[7]。本次比赛中，我们选取了5种特征，详细描述见表4。其中角度特征的抽取我们借用了[AngNet](https://arxiv.org/abs/2105.01563)[9]中对关节点之间的角度的设计，由于本数据集没有手部关节点，所以我们只利用了前7种Angle，如图5所示:


<p align="center">
<img src="https://ai-studio-static-online.cdn.bcebos.com/4e62ea43ce264caa86625e573ccab532d1aba15c5f804a30bc0cfc962ed0d556" width = "800" alt="" align="center" />
</p><br><center>图5 Angle特征示意图</center></br>

从以上5种特征出发，我们产生了以下7种特征组合分别作为模型的输入，每一种组合对应一个以不同特征为输入的CTR-GCN模型，最终结果由表5中列举的7种模型进行集成得到。

<div align="left">
  <br>表4 使用的特征及描述</br>
    <table>
        <tr>
            <th>特征</th>
            <th>描述</th>
        </tr>
        <tr>
            <td>Joint</td>
            <td>原始关节点坐标</td>
        </tr>
        <tr>
            <td>Bone</td>
            <td>由源关节点指向汇关节点的二维向量，代表骨骼信息</td>
        </tr>
        <tr>
            <td>Angles</td>
            <td>骨骼的夹角信息，共有7种夹角，详见代码说明部分</td>
        </tr>
        <tr>
            <td>Joint Motion</td>
            <td>同一关节点坐标在相邻帧间的差值</td>
        </tr>
        <tr>
            <td>Bone Motion</td>
            <td>同一骨骼向量在相邻帧间的差值</td>
        </tr>
    </table>
</div>

<div>
	<br>表5 7种模型使用的输入特征组合及维度</br>
</div>

|特性组合 (缩写)|特征维度|
|-------|----|
|Joint ($J$)|2|
|Bone ($B$)|2|
|Joint + Angle ($JA$)|9|
|Bone + Angle ($BA$)|9|
|Joint + Bone ($JB$)|4|
|Joint + Joint Motion ($JM_j$)|4|
|Bone Motion ($M_b$)|2|

## 参考资料

[1] Chen, Yuxin, et al. "Channel-wise Topology Refinement Graph Convolution for Skeleton-Based Action Recognition." Proceedings of the IEEE/CVF International Conference on Computer Vision. 2021.

[2] Lin, Tsung-Yi, et al. "Focal loss for dense object detection." Proceedings of the IEEE international conference on computer vision. 2017.

[3] Huang, Zhen, et al. "Spatio-temporal inception graph convolutional networks for skeleton-based action recognition." Proceedings of the 28th ACM International Conference on Multimedia. 2020.

[4] https://github.com/PaddlePaddle/PaddleVideo/blob/develop/docs/zh-CN/model_zoo/recognition/agcn.md

[5] Duan, Haodong, et al. "Revisiting Skeleton-based Action Recognition." arXiv preprint arXiv:2104.13586 (2021).

[6] Loshchilov, Ilya, and Frank Hutter. "Sgdr: Stochastic gradient descent with warm restarts." arXiv preprint arXiv:1608.03983 (2016).

[7] Shi, Lei, et al. "Skeleton-based action recognition with multi-stream adaptive graph convolutional networks." IEEE Transactions on Image Processing 29 (2020): 9532-9545.

[8] https://github.com/Uason-Chen/CTR-GCN

[9] Qin, Zhenyue, et al. "Fusing Higher-Order Features in Graph Neural Networks for Skeleton-Based Action Recognition." arXiv preprint arXiv:2105.01563 (2021).

# 模型训练与预测代码说明

## 目录结构

```text
├── aistudio_project.ipynb
├── data
│   ├── test_A_data.npy
│   ├── test_B_data.npy
│   ├── train_data.npy
│   └── train_label.npy
└── work
    ├── dataset
    │   ├── generate_feature.py
    │   ├── generate_index.py
    │   └── index
    ├── doc
    └── PaddleVideo
        ├── configs
        ├── ensemble.py
        ├── LICENSE
        ├── logits
        ├── main.py
        ├── model_ours
        ├── paddlevideo
        ├── README_cn.md
        ├── README.md
        ├── requirements.txt
        ├── run_test.sh
        └── run_train.sh
```

## 目录与文件描述

|目录/文件名|描述|
|---|---|
|aistudio_project.ipynb|AIStudio 项目的Notebook，包含项目介绍与运行方式|
|data|data目录用于存放数据集，需自行下载数据后按目录结构进行组织|
|work/dataset/generate_feature.py| 数据预处理脚本，用于特征抽取 |
|work/dataset/generate_index.py| 数据预处理脚本，用于划分验证集|
|work/dataset/index|该目录下存储指示验证集的划分方式的npy文件|
|work/PaddleVideo/configs| 该目录下存储模型的yaml配置文件|
|work/PaddleVideo/ensenmble.py|融合模型预测结果的脚本|
|work/PaddleVideo/logits|该目录用于存放模型的预测结果|
|work/PaddleVideo/model_ours|该目录下存放训练完毕的模型，用于复现B榜成绩|
|work/PaddleVideo/requirements.txt|依赖配置文件|
|work/PaddleVideo/run_test.sh|模型预测脚本，用于复现B榜成绩|
|work/PaddleVideo/run_train.sh|模型训练脚本|

下面我们介绍代码的运行过程

In [ ]:
# 配置代码环境，安装相应的依赖包
%cd ~/work/PaddleVideo/
!python3.7 -m pip install --upgrade pip
!python3.7 -m pip install --upgrade -r requirements.txt

## 数据预处理
数据预处理由三部分组成：五折交叉验证集的生成、特征提取与训练数据上采样。

### 1. 五折交叉验证
由于本数据集未给出验证集，所以我们按照训练集和验证集8:2的比例对数据集进行了的五折划分，通过`~/work/dataset/generate_index.py`生成各个fold的训练和验证集的index。我们生成的index在`~/work/dataset/index/`中已经给出，不需要重新生成。

### 2. 特征提取
由于数据集本身的信息量较小，不能使模型得到很好地训练，对此我们根据原始数据生成了7种不同的特征，包括J,JA,JMj,B,BA,JB,Mb，其中J代表关节点(Joints)，B代表关节向量(Bone)，A代表关节之间的角度(Angle)，Mj代表J的运动向量(Joints Motion)，Mb代表B的运动向量(Bone Motion)。

### 3. 数据上采样
我们统计了数据集中所有类别对应的样本数目，发现类别之间存在样本数量不均衡问题(最多相差6.7倍)，这会导致样本数量较少的类别无法得到充分的学习。对此我们使用Upsampling，对我们划分好的训练集进行上采样，使每个类别的样本数目保持一致。验证集和测试集不需要Upsampling。

### 数据预处理命令汇总

我们将以上数据预处理步骤统一整合至generate_feature.py文件中，生成数据时只需调用该python脚本即可。

注意事项
- 通过`-m`参数指定生成数据类型（train/valid/test），默认生成测试集的数据。
- 生成的数据将分别保存在`~/work/dataset/train/`、`~/work/dataset/valid/`和`~/work/dataset/test/`中
- 训练集部分的数据生成花费时间较长且占用内存较多（需至少xGB的内存才可正常运行，耗时约60分钟）。我们已经提供了训练完毕的模型参数，如需复现B的结果，只需生成test部分的数据

```bash
cd ~/work/dataset/
# generate train data
python3.7 generate_feature.py -m train

# generate validation data
python3.7 generate_feature.py -m valid

# generate test data
python3.7 generate_feature.py -m test
```

In [ ]:
# generate test data
%cd ~/work/dataset/
!python3.7 generate_feature.py -m test

/home/aistudio/work/dataset
=========>> JA finished!!! <<=========
=========>> J finished!!! <<=========
=========>> BA finished!!! <<=========
=========>> B finished!!! <<=========
=========>> JB finished!!! <<=========
=========>> JMj finished!!! <<=========
=========>> Mb finished!!! <<=========


## 训练脚本
按照上述步骤进行数据生成后，可使用本模块描述的训练脚本进行模型训练。

### 单一模型训练启动命令
使用以下脚本可以完成单一模型的训练。

注意事项：
- 通过`-c`参数指定配置文件。训练完毕后，验证集上准确率最高的模型将保存在`~/work/model/`路径下。比赛中我们使用的全部模型的yaml配置文件均已存放在./configs/recognition/ctrgcn目录下。

```bash
cd ~/work/PaddleVideo/
python3.7 main.py -c $PATH_TO_YOUR_CONFIGURATIOM_YAML --validate
# 运行示例: python3.7 main.py -c configs/recognition/ctrgcn/ctrgcn_fsd_J_fold0.yaml --validate
```

### 集成模型训练启动命令
由于本队的最终模型由一系列模型的集成得到，手动配置yaml文件训练逐一模型需要耗费大量的精力。这里我们将本队使用的全部模型的训练命令整合至run_train.sh文件下，如需完整复现，只需运行以下命令，无需手动配置yaml文件。

注意事项：
- 需要注意的是，由于模型数量较多，该命令的运行需要耗费较长时间（约2.5h * 35 = 87.5h) 

```bash
cd ~/work/PaddleVideo/
sh run_train.sh
```

## 测试脚本
模型训练完成后，可使用本节所描述的测试脚本进行评估。

### 单一模型测试启动命令
使用以下命令可以完成单一模型在相应测试集上的推理，推理结果将保存在`~/work/PaddleVideo/submission.csv`文件中。

注意事项：
- 通过`-c`参数指定配置文件，通过`-w`指定权重存放路径进行模型测试。

```bash
cd ~/work/PaddleVideo/
python3.7 main.py --test -c $PATH_TO_YOUR_CONFIGURATIOM_YAML -w $PAHT_TO_YOUR_PDPARAMS_FILE
```


### 集成模型测试启动命令

由于完整的模型测试需要综合35个模型进行集成，配置较为复杂，这里我们将35个模型的推理与集成命令整合至run_test.sh文件中，如需复现结果只需运行如下命令即可。得到的评估结果保存在`~/work/PaddleVideo/submission_ensemble.csv`文件中。

注意事项:
- 如需在您训练得到的模型上进行测试，您需要修改run_test.sh文件中配置的模型文件路径

```bash
cd ~/work/PaddleVideo/
sh run_test.sh
```


In [7]:
%cd ~/work/PaddleVideo/
!sh run_test.sh

/home/aistudio/work/PaddleVideo
[11/25 17:00:05] DALI is not installed, you can improve performance if use DALI
[11/25 17:00:05] DATASET : 
[11/25 17:00:05]     batch_size : 16
[11/25 17:00:05]     num_workers : 2
[11/25 17:00:05]     test : 
[11/25 17:00:05]         file_path : /home/aistudio/work/dataset/test/J.npy
[11/25 17:00:05]         format : SkeletonDataset
[11/25 17:00:05]         test_mode : True
[11/25 17:00:05]     test_batch_size : 1
[11/25 17:00:05]     test_num_workers : 0
[11/25 17:00:05]     train : 
[11/25 17:00:05]         file_path : /home/aistudio/work/dataset/train/J_fold0.npy
[11/25 17:00:05]         format : SkeletonDataset
[11/25 17:00:05]         label_path : /home/aistudio/work/dataset/train/fold0_label.npy
[11/25 17:00:05]     valid : 
[11/25 17:00:05]         file_path : /home/aistudio/work/dataset/valid/J_fold0.npy
[11/25 17:00:05]         format : SkeletonDataset
[11/25 17:00:05]         label_path : /home/aistudio/work/dataset/valid/fold0_label.npy
[11/

请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 